

####Climate Risk Modeling involves predicting how climate change or extreme weather events (like floods or hurricanes) could impact financial assets. For example, if a flood damages properties, loans given to those property owners might be at higher risk of default.


Credit Risk is the risk that a borrower will not repay their loan. Financial institutions need to estimate this risk to manage their portfolios effectively.
Simple Example
Imagine you work at a bank, and you want to see how a major storm might affect your loans. You have a bunch of loans, and each loan has a chance of defaulting (not being paid back) which you measure as the Probability of Default (PD).

####Steps to Model the Impact of Climate Risk:

Generate Loan Data:
* Create a list of loans with their amounts and the initial chance of default.


Define a Stress Scenario:

* Assume that a major storm increases the chance of default for these loans. For instance, we might say that the probability of default increases by 25%.

Adjust PD for Stress Scenario:

* Apply this increase to the initial chance of default to get a new, higher chance of default.

Simulate Defaults:

* Use the new chance of default to predict which loans will actually default.

Calculate Expected Loss:

* Estimate how much money we might lose if these defaults occur, considering that we can recover some money from the defaulted loans.


In [1]:
import numpy as np
import pandas as pd

# Step 1: Generate Synthetic Loan Data
np.random.seed(42)  # For reproducibility
loan_data = pd.DataFrame({
    'loan_id': range(1, 101),  # Loan IDs from 1 to 100
    'loan_amount': np.random.uniform(10000, 100000, 100),  # Random loan amounts between $10,000 and $100,000
    'initial_pd': np.random.uniform(0.01, 0.10, 100)  # Initial chance of default between 1% and 10%
})
loan_data.head()



,loan_id,loan_amount,initial_pd
0,1,43708.610696,0.012829
1,2,95564.287577,0.067277
2,3,75879.454763,0.038292
3,4,63879.263578,0.055771
4,5,24041.677640,0.091681


In [2]:
# Step 2: Define Climate Stress Scenario
# Increase the PD by 25% due to extreme weather
def apply_climate_stress_scenario(pd_values, increase_factor=0.25):
    return np.minimum(pd_values * (1 + increase_factor), 1.0)  # Ensure PD does not exceed 100%

# Apply the stress scenario
loan_data['stressed_pd'] = apply_climate_stress_scenario(loan_data['initial_pd'])

loan_data.head()

,loan_id,loan_amount,initial_pd,stressed_pd
0,1,43708.610696,0.012829,0.016036
1,2,95564.287577,0.067277,0.084096
2,3,75879.454763,0.038292,0.047865
3,4,63879.263578,0.055771,0.069714
4,5,24041.677640,0.091681,0.114601


In [3]:
# Step 3: Simulate Defaults
# Use the stressed PD to simulate which loans default
def simulate_defaults(pd_values):
    return np.random.binomial(1, pd_values)  # 1 for default, 0 for no default

loan_data['default_event'] = simulate_defaults(loan_data['stressed_pd'])
loan_data

,loan_id,loan_amount,initial_pd,stressed_pd,default_event
0,1,43708.610696,0.012829,0.016036,0
1,2,95564.287577,0.067277,0.084096,0
2,3,75879.454763,0.038292,0.047865,0
3,4,63879.263578,0.055771,0.069714,0
4,5,24041.677640,0.091681,0.114601,0
...,...,...,...,...,...
95,96,54441.603673,0.041429,0.051786,0
96,97,57045.954644,0.075336,0.094170,0
97,98,48478.691652,0.090740,0.113425,0
98,99,12287.721407,0.089838,0.112297,0


In [4]:
# Step 4: Calculate Expected Loss
# Assume we can recover 40% of the loan amount, so LGD (Loss Given Default) is 60%
recovery_rate = 0.40
loan_data['lgd'] = 1 - recovery_rate

# Calculate expected loss for each loan
loan_data['expected_loss'] = round( loan_data['loan_amount'] * loan_data['initial_pd'] * loan_data['lgd'])
loan_data['expected_loss_sttressed'] = round( loan_data['loan_amount'] * loan_data['stressed_pd'] * loan_data['lgd'])
loan_data.head()

,loan_id,loan_amount,initial_pd,stressed_pd,default_event,lgd,expected_loss,expected_loss_sttressed
0,1,43708.610696,0.012829,0.016036,0,0.6,336.0,421.0
1,2,95564.287577,0.067277,0.084096,0,0.6,3858.0,4822.0
2,3,75879.454763,0.038292,0.047865,0,0.6,1743.0,2179.0
3,4,63879.263578,0.055771,0.069714,0,0.6,2138.0,2672.0
4,5,24041.677640,0.091681,0.114601,0,0.6,1322.0,1653.0


In [5]:


# Summarize results
total_expected_loss = loan_data['expected_loss'].sum()
total_expected_loss_stressed = loan_data['expected_loss_sttressed'].sum()
total_default_loans = loan_data['default_event'].sum()

print(f"Total Expected Loss under Climate Stress Scenario: ${total_expected_loss:.2f}")
print(f"Total Expected Loss Stressed under Climate Stress Scenario: ${total_expected_loss_stressed:.2f}")
print(f"Total Number of Default Loans: {total_default_loans}")

# Display sample data
print("\nSample Loan Data with Stressed PD and Expected Loss:")
loan_data[loan_data['default_event']==1]

Total Expected Loss under Climate Stress Scenario: $170602.00
Total Expected Loss Stressed under Climate Stress Scenario: $213254.00
Total Number of Default Loans: 5

Sample Loan Data with Stressed PD and Expected Loss:


,loan_id,loan_amount,initial_pd,stressed_pd,default_event,lgd,expected_loss,expected_loss_sttressed
26,27,27970.640394,0.083621,0.104527,1,0.6,1403.0,1754.0
40,41,20983.441136,0.096620,0.120775,1,0.6,1216.0,1521.0
47,48,56806.121906,0.055241,0.069051,1,0.6,1883.0,2354.0
56,57,17964.325185,0.070492,0.088115,1,0.6,760.0,950.0
61,62,34421.412860,0.066908,0.083634,1,0.6,1382.0,1727.0


##Stress Testing a Portfolio of Mortgage-Backed Securities

Here’s an example of how you might develop a stress test scenario for a portfolio of MBS during an economic downturn. Suppose we have a portfolio of mortgage-backed securities (MBS). During an economic downturn, factors such as increasing unemployment rates and falling housing prices can impact the performance of these securities. By developing a stress test scenario that simulates such adverse conditions, we can identify the vulnerabilities within the portfolio and adjust our risk strategies accordingly.



In [6]:
import numpy as np
import pandas as pd

# Sample data: Portfolio of mortgage-backed securities
data = {
    'MBS_ID': ['MBS1', 'MBS2', 'MBS3', 'MBS4', 'MBS5'],
    'Market_Value': [1000000, 2000000, 1500000, 1200000, 1800000],
    'Credit_Rating': ['AAA', 'AA', 'A', 'BBB', 'BB'],
    'Default_Probability': [0.01, 0.02, 0.03, 0.05, 0.1]  # Base probabilities
}

portfolio = pd.DataFrame(data)

# Economic downturn scenario: Increase default probabilities
scenario_factor = 3  # Assuming default probabilities triple during the downturn
portfolio['Stress_Default_Probability'] = portfolio['Default_Probability'] * scenario_factor

# Estimate potential losses
loss_given_default = 0.4  # Assuming 40% loss given default
portfolio['Expected_Loss'] = portfolio['Market_Value'] * portfolio['Stress_Default_Probability'] * loss_given_default

# Calculate total expected loss for the portfolio
total_expected_loss = portfolio['Expected_Loss'].sum()

# Output the results
print("Portfolio under Stress Scenario:")
portfolio


Portfolio under Stress Scenario:


,MBS_ID,Market_Value,Credit_Rating,Default_Probability,Stress_Default_Probability,Expected_Loss
0,MBS1,1000000,AAA,0.01,0.03,12000.0
1,MBS2,2000000,AA,0.02,0.06,48000.0
2,MBS3,1500000,A,0.03,0.09,54000.0
3,MBS4,1200000,BBB,0.05,0.15,72000.0
4,MBS5,1800000,BB,0.10,0.30,216000.0


In [7]:
print(f"\nTotal Expected Loss under Stress Scenario: ${total_expected_loss:,.2f}")

# Adjust risk strategies based on the stress test results
# For example, rebalancing the portfolio to reduce exposure to higher-risk securities
high_risk_securities = portfolio[portfolio['Stress_Default_Probability'] > 0.1]

print("\nHigh-Risk Securities under Stress Scenario:")
high_risk_securities



Total Expected Loss under Stress Scenario: $402,000.00

High-Risk Securities under Stress Scenario:


,MBS_ID,Market_Value,Credit_Rating,Default_Probability,Stress_Default_Probability,Expected_Loss
3,MBS4,1200000,BBB,0.05,0.15,72000.0
4,MBS5,1800000,BB,0.10,0.30,216000.0
